In [1]:
import sys

from quasinet.qnet import qdistance
from quasinet.qnet import load_qnet
import pandas as pd
import numpy as np
from tqdm import tqdm

import sys

from quasinet.qnet import load_qnet
import seaborn as sns
from quasinet.qnet import export_qnet_tree
import graph_tool.all as gt
from matplotlib import cm
import glob

In [2]:
import zipfile
import os
with zipfile.ZipFile('../north_h1n1_ha_20.joblib.zip', 'r') as zip_ref:
    zip_ref.extractall('./')
qnet=load_qnet('./north_h1n1_ha_20.joblib')
os.remove('./north_h1n1_ha_20.joblib')

In [3]:
from quasinet.qnet import export_qnet_graph
dotfile='qgraph.dot'
export_qnet_graph(qnet,threshold=0.08,outfile=dotfile)


In [4]:
def getLoops(DOTNAME,VERBOSE=False,suffix='',STRONG_COMPONENT=True,varnames=None):
    g = gt.load_graph(DOTNAME)
    if VERBOSE:
        for v in g.vertices():
            print('deg',v.out_degree())
        g.list_properties()

    if STRONG_COMPONENT:
        g=gt.extract_largest_component(
                    g, directed=True, prune=True)
    C=gt.all_circuits(g)
    
    for count,c in enumerate(C):
        a=np.intersect1d(c,varinds)
        if len(a) > 0:
            break

    return vars,varinds,c

def getShortestPath2andF(DOTNAME,src=None,tgt=None,VERBOSE=False,suffix='',varnames=None):
    '''
        get shortest path 
    '''
    g = gt.load_graph(DOTNAME)
    g=gt.extract_largest_component(
                    g, directed=True, prune=True)
    names=np.array([g.vp.vertex_name[v] for v in g.vertices()])
    
    src=np.where(names==src)[0][0]
    tgt=np.where(names==tgt)[0][0]
    
    vlist, elist = gt.shortest_path(g, g.vertex(src), g.vertex(tgt))
    vlist1, elist1 = gt.shortest_path(g, g.vertex(tgt), g.vertex(src))
    
    a=np.array([int(str(v)) for v in vlist])
    b=np.array([int(str(v)) for v in vlist1])
    c=np.hstack((a[1:],b[1:]))
    
    unq=False
    if len(list(c)) == len(list(set(list(c)))):
        unq=True
    
    return c,names[c],len(c),unq


def getvarsStronglyconnected(dotfile,POLEFILE=None):
    '''
        return variables in the strongly connected component, and the 
        name index map
        if POLEFILE is set, return only vars from the poles
    '''
    g = gt.load_graph(dotfile)
    g=gt.extract_largest_component(
                    g, directed=True, prune=True)
    names=np.array([g.vp.vertex_name[v] for v in g.vertices()])
    vars=pd.DataFrame(names,columns=['var']).reset_index().set_index('var').transpose()
    if POLEFILE is not None:
        polevar=pd.read_csv(POLEFILE,index_col=0).index
        polevars_=[x for x in polevar if x in vars]
        vars=vars[polevars_]
    vars=vars.transpose()
    vars.columns=['ind']
        
    namemap=pd.DataFrame(names,columns=['nm'])

    return vars,namemap

col={'soc':'IndianRed1!50','reli':'SeaGreen2!50',
     'pol':'DodgerBlue4!50',
     '':'white',
     'edu':'Yellow2!70','soc2':'IndianRed1!50',
     'soc3':'IndianRed1!50','mark':'Orchid1'}
def genFig(dotfile,name1,name2,suffix='x',RAD=4,namemap=None,col=None):

    def namemap_(ind):
        if namemap is None:
            return ''
        if ind in namemap.keys():
            return namemap[ind]
        else:
            if 'con' in ind[:3]:
                return 'pol'
            if 'reli' in ind[:4]:
                return 'reli'
            
            return ''
    
    path_,pathname_,L,unq=getShortestPath2andF(dotfile,src=name1,tgt=name2)
    print(path_,pathname_)
    a=pathname_
    
    if namemap is not None:
        vartype=[col[namemap_(i)] for i in pathname_]
            
    
    #print(namemap.vartype.values[path_])
    if unq:
        amat=None
        for i in range(len(a)-1):
            if amat is None:
                amat=[a[i],a[i+1]]
            else:
                amat=np.vstack((amat,[a[i],a[i+1]]))
        amat=np.vstack((amat,[a[-1],a[0]]))
        
        pd.DataFrame(amat,columns=['u','v']).assign(Direct='true').to_csv('edges'+suffix+'.csv',index=None)
        vertices=pd.DataFrame(pathname_,columns=['id'])
        n=vertices.index.size
        RAD=RAD
        x=[np.round(RAD*np.cos((i/n)*2*np.pi),2) for i in range(n)]
        y=[np.round(-RAD*np.sin((i/n)*2*np.pi),2) for i in range(n)]
        vertices=vertices.assign(label=vertices.id,x=x,y=y)
        if namemap is not None:
            vertices=vertices.assign(color=vartype)
        vertices.to_csv('vertices'+suffix+'.csv',index=None)
        return
    
    print(np.unique(path_, return_counts=True))
    print('not a true cycle')
    return a,namemap.vartype.values

In [5]:
vars,ddf=getvarsStronglyconnected(dotfile,POLEFILE=None)
ddf=np.array([x[0] for x in ddf.values])

In [6]:
getShortestPath2andF(dotfile,src='x304',tgt='x223')

(array([30, 22, 12, 53, 38, 27, 37]),
 array(['x278', 'x223', 'x155', 'x63', 'x318', 'x274', 'x304'], dtype='<U4'),
 7,
 True)

In [7]:
for i in np.arange(len(ddf)):
    for j in np.arange(0,i):
        a,b,c,d=getShortestPath2andF(dotfile,src=ddf[i],tgt=ddf[j])
        if  d and c<5:
            print(ddf[i],ddf[j],c,b)
        

x155 x14 4 ['x63' 'x14' 'x223' 'x155']
x158 x144 3 ['x144' 'x337' 'x158']
x171 x141 3 ['x201' 'x141' 'x171']
x179 x158 3 ['x158' 'x299' 'x179']
x201 x141 3 ['x141' 'x171' 'x201']
x201 x171 3 ['x141' 'x171' 'x201']
x213 x212 2 ['x212' 'x213']
x223 x14 2 ['x14' 'x223']
x223 x155 4 ['x155' 'x63' 'x14' 'x223']
x250 x15 2 ['x15' 'x250']
x299 x158 2 ['x158' 'x299']
x299 x179 3 ['x179' 'x158' 'x299']
x337 x144 3 ['x158' 'x144' 'x337']
x337 x158 2 ['x158' 'x337']
x386 x287 2 ['x287' 'x386']
x392 x10 2 ['x10' 'x392']
x456 x201 2 ['x201' 'x456']
x467 x10 4 ['x47' 'x10' 'x392' 'x467']
x467 x113 2 ['x113' 'x467']
x467 x392 4 ['x47' 'x10' 'x392' 'x467']
x47 x10 4 ['x10' 'x392' 'x467' 'x47']
x47 x392 4 ['x10' 'x392' 'x467' 'x47']
x47 x467 4 ['x10' 'x392' 'x467' 'x47']
x490 x299 2 ['x299' 'x490']
x52 x211 2 ['x211' 'x52']
x525 x2 2 ['x2' 'x525']
x525 x277 2 ['x277' 'x525']
x63 x123 2 ['x123' 'x63']
x63 x14 4 ['x14' 'x223' 'x155' 'x63']
x63 x155 4 ['x14' 'x223' 'x155' 'x63']
x63 x223 4 ['x14' 'x223' '

In [8]:
genFig(dotfile,'x223','x155',suffix='x',RAD=4,namemap=None,col=None)

[12 53  6 22] ['x155' 'x63' 'x14' 'x223']
